In [1]:
# Set tokenizer and build vocabulary
from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

en_tokenizer = get_tokenizer('spacy', language='en_core_web_md')
de_tokenizer = get_tokenizer('spacy', language='de_core_news_md')

def yield_tokens(data_iter, language: str):
    for text in data_iter:
        if language == 'en':
            yield en_tokenizer(text[0])
        elif language == 'de':
            yield de_tokenizer(text[1])

UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_tokens = ['<unk>', '<pad>', '<bos>', '<eos>']
en_vocab = build_vocab_from_iterator(yield_tokens(Multi30k(split='train', language_pair=('en', 'de')), 'en'), specials=special_tokens, special_first=True, min_freq=3)
de_vocab = build_vocab_from_iterator(yield_tokens(Multi30k(split='train', language_pair=('en', 'de')), 'de'), specials=special_tokens, special_first=True, min_freq=3)
en_vocab.set_default_index(UNK_IDX) # oov 일때 반환하는 토큰
de_vocab.set_default_index(UNK_IDX)

/Users/wonjun/Desktop/projects/AI-Paper-Reproduce/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set preprocess pipeline
en_pipeline = lambda x: en_vocab(en_tokenizer(x))
de_pipeline = lambda x: de_vocab(de_tokenizer(x))
print(en_pipeline('Several men in hard hats are operating a giant pulley system.'))
print(de_pipeline('Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.'))

[165, 36, 7, 335, 287, 17, 1224, 4, 758, 4496, 2957, 5]
[84, 31, 10, 847, 2208, 15, 0, 4]


In [3]:
from torch.utils.data import DataLoader

train_iter = Multi30k(split='train', language_pair=('en', 'de'))
train_loader = DataLoader(train_iter, batch_size=2)
for src, data in train_loader:
    pass

In [19]:
# Define Transformer
# I follow style of official Pytorch Transformer source code and adjust it simply
import copy
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn import ModuleList
from torch.nn.init import xavier_uniform_
from torch.nn.modules import LayerNorm
from torch.nn.parameter import Parameter
import pytorch_lightning as pl
from typing import Union, Callable, Optional, Any, Tuple


class TransformerWrapper(pl.LightningModule):
    def __init__(self, transformer):
        self.model = transformer
    def training_step(self, *args: Any, **kwargs: Any):
        return super().training_step(*args, **kwargs)
    def validation_step(self, *args: Any, **kwargs: Any):
        return super().validation_step(*args, **kwargs)
    def configure_optimizers(self) -> Any:
        return super().configure_optimizers()


class Transformer(nn.Module):
    def __init__(self, d_model: int = 512, n_head: int = 8, num_encoder_layers: int = 6, num_decoder_layers: int = 6, \
        dim_feedforward: int = 2048, dropout: float = 0.1, activation: Union[str, Callable[[Tensor], Tensor]] = F.relu, \
        layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False, device=None, dtype=None) -> None:
        
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(Transformer, self).__init__()
        # TODO 
        encoder_layer = TransformerEncoderLayer(d_model, n_head, dim_feedforward, dropout, activation, layer_norm_eps, batch_first, norm_first, **factory_kwargs)
        encoder_norm = LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        decoder_layer = None
        decoder_norm = None
        self.decoder = None

        self._reset_params()

        self.d_model = d_model
        self.n_head = n_head
        self.batch_first = batch_first
    
    def forward(self, src: Tensor, tgt: Tensor, src_mask: Optional[Tensor] = None, tgt_mask: Optional[Tensor] = None, \
                memory_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None, tgt_key_padding_mask: Optional[Tensor] = None, \
                memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:

                memory = self.encoder()
                output  = self.decoder()
                return output
    
    def _reset_params(self):
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)


class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
    
    def forward(self, src: Tensor, mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        output = src
        for mod in self.layers:
            output = mod(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
        
        if self.norm is not None:
            output = self.norm(output)
        
        return output

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model: int, n_head: int, dim_feedforward: int = 2048, dropout: float = 0.1, \
                activation: Union[str, Callable[[Tensor], Tensor]] = F.relu, layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,  
                device=None, dtype=None) -> None:

        factory_kwargs = {'device': device, 'dtype': dtype}
        super(TransformerEncoderLayer, self).__init__()
        
        # Define params
        self.self_attn = MultiheadAttention(d_model, n_head, dropout, batch_first, **factory_kwargs)
        self.linear1 = nn.Linear(d_model, dim_feedforward, **factory_kwargs)
        self.droput = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model, **factory_kwargs)
        
        self.norm_first = norm_first
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        self.activation = activation
    
    def forward(self, src: Tensor, src_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        x = src
        if self.norm_first:
            x = x + self._self_attn_block(self.norm1(src), src_mask, src_key_padding_mask)
            x = x + self._feedforward_block(self.norm2(x))
        else:
            x = self.norm1(x + self._self_attn_block(x, src_mask, src_key_padding_mask))
            x = self.norm2(x + self._feedforward_block(x))
        return x

    def _self_attn_block(self, x: Tensor, attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.self_attn(x, x, x, attn_mask=attn_mask, key_padding_mask=key_padding_mask, need_weights=False)[0]
        return self.dropout1(x)
    
    def _feedforward_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout2(self.activation(self.linear1(x))))
        return self.dropout2(x)


class MultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0, batch_first=False, device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.batch_first = batch_first
        self.head_dim = int(embed_dim // num_heads)
        assert self.embed_dim == self.head_dim * num_heads

        # self.in_proj_weight = Parameter(torch.empty((3*embed_dim, embed_dim), **factory_kwargs))
        # self.register_parameter('q_proj_weight', None)
        # self.register_parameter('k_proj_weight', None)
        # self.register_parameter('v_proj_weight', None)

        self.wq = nn.Linear(embed_dim, embed_dim, bias=False, **factory_kwargs)
        self.wk = nn.Linear(embed_dim, embed_dim, bias=False, **factory_kwargs)
        self.wv = nn.Linear(embed_dim, embed_dim, bias=False, **factory_kwargs)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=False, **factory_kwargs)

    def forward(self, query: Tensor, key: Tensor, value: Tensor, key_padding_mask: Optional[Tensor] = None, need_weights: bool = True, \
                attn_mask: Optional[Tensor] = None, average_attn_weights: bool = True) -> Tuple[Tensor, Optional[Tensor]]:

        query, key, value = self.wq(query), self.wk(key), self.wv(value)
        query, key ,value = self._split_heads(query), self._split_heads(key), self._split_heads(value)
        attn_out = self._attention(query, key, value)
        attn_out = self.out_proj(attn_out)
        return attn_out
    
    def _split_heads(self, proj: Tensor) -> Tensor:
        if self.batch_first:    # (N, L, E)
            bs = proj.size(0)
            proj = proj.view(bs, -1, self.num_heads, self.head_dim)    # (N, L, H, E_Hi)
            proj = proj.transpose(1, 2) # (N, H, L, E_Hi)
        else:   # (L, N, E)
            bs = proj.size(1)
            proj = proj.view(-1, bs, self.num_heads, self.head_dim)    # (L, N, H, E_Hi)
        return proj
    
    def _attention(self, query: Tensor, key: Tensor, value: Tensor) -> Tensor:
        if self.batch_first:
            score = torch.matmul(query, key.transpose(-1, -2))  # (N, H, L, L), score = Q * K^T
            score = score / math.sqrt(query.size(-1)) # score = Q * K^T divided by sqrt(E_Hi)
            softmax = F.softmax(score, dim=-1)
            attn_out = torch.matmul(score, value)   # (N, H, L, E_Hi)
            attn_out = attn_out.transpose(1, 2) # (N, L, H, E_Hi)
            attn_out = attn_out.contiguous().view(attn_out.size(0), -1, self.embed_dim)  # (N, L, E)
            return attn_out
        else:
            raise NotImplementedError()


def _get_clones(module, N):
    return ModuleList([copy.deepcopy(module) for i in range(N)])


In [21]:
dummy_src = torch.rand((5, 10, 512))
d_model = 512
n_head = 8
encoder_layer = TransformerEncoderLayer(d_model, n_head, batch_first=True)
encoder = TransformerEncoder(encoder_layer, num_layers=6)
out = encoder(dummy_src)
encoder.parameters
# commit test

<bound method Module.parameters of TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (wq): Linear(in_features=512, out_features=512, bias=False)
        (wk): Linear(in_features=512, out_features=512, bias=False)
        (wv): Linear(in_features=512, out_features=512, bias=False)
        (out_proj): Linear(in_features=512, out_features=512, bias=False)
      )
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (droput): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=512, bias=True)
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (wq): Linear(in_features=512, out_features=512, bias=